In [1]:
from google.colab import drive
drive.mount('/content/gdrive/')

Mounted at /content/gdrive/


In [2]:
!pip install kaggle
!mkdir ~/.kaggle
!cp /content/gdrive/MyDrive/kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

In [3]:
!kaggle datasets list

ref                                                         title                                              size  lastUpdated          downloadCount  
----------------------------------------------------------  ------------------------------------------------  -----  -------------------  -------------  
gpreda/reddit-vaccine-myths                                 Reddit Vaccine Myths                              227KB  2021-05-06 19:17:06           4466  
crowww/a-large-scale-fish-dataset                           A Large Scale Fish Dataset                          3GB  2021-04-28 17:03:01           2563  
promptcloud/careerbuilder-job-listing-2020                  Careerbuilder Job Listing 2020                     42MB  2021-03-05 06:59:52            607  
mathurinache/twitter-edge-nodes                             Twitter Edge Nodes                                342MB  2021-03-08 06:43:04            274  
dhruvildave/wikibooks-dataset                               Wikibooks Datase

In [4]:
import os
import shutil
import cv2
import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras import Model
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import Adam

In [5]:
%%capture
!kaggle datasets download -d tawsifurrahman/covid19-radiography-database
!mkdir covid19_radiography_data
!unzip covid19-radiography-database.zip -d covid19_radiography_data

In [6]:
data_dir_covid = '/content/covid19_radiography_data/COVID-19_Radiography_Dataset/COVID/'
data_dir_normal = '/content/covid19_radiography_data/COVID-19_Radiography_Dataset/Normal/'

allfiles_covid = [f for f in os.listdir(data_dir_covid) if os.path.isfile(os.path.join(data_dir_covid, f))]
allfiles_normal = [f for f in os.listdir(data_dir_normal) if os.path.isfile(os.path.join(data_dir_normal, f))]
len(allfiles_covid)

3616

In [7]:
!unzip -u "/content/gdrive/My Drive/Colab Notebooks/generated_imgs.zip" -d '/content/covid19_radiography_data/'

Archive:  /content/gdrive/My Drive/Colab Notebooks/generated_imgs.zip
  inflating: /content/covid19_radiography_data/generated_imgs/covid/990.png  
  inflating: /content/covid19_radiography_data/generated_imgs/covid/995.png  
  inflating: /content/covid19_radiography_data/generated_imgs/covid/989.png  
  inflating: /content/covid19_radiography_data/generated_imgs/covid/998.png  
  inflating: /content/covid19_radiography_data/generated_imgs/covid/988.png  
  inflating: /content/covid19_radiography_data/generated_imgs/covid/991.png  
  inflating: /content/covid19_radiography_data/generated_imgs/covid/992.png  
  inflating: /content/covid19_radiography_data/generated_imgs/covid/996.png  
  inflating: /content/covid19_radiography_data/generated_imgs/covid/985.png  
  inflating: /content/covid19_radiography_data/generated_imgs/covid/997.png  
  inflating: /content/covid19_radiography_data/generated_imgs/covid/975.png  
  inflating: /content/covid19_radiography_data/generated_imgs/covid/993.

In [8]:
!ls '/content/covid19_radiography_data/generated_imgs/'

covid  normal


In [9]:
generated_data_dir = '/content/covid19_radiography_data/generated_imgs/'
generated_covid_dir = os.path.join(generated_data_dir,'covid')
generated_normal_dir = os.path.join(generated_data_dir,'normal')
generated_covid_files = [f for f in os.listdir(generated_covid_dir) if os.path.isfile(os.path.join(generated_covid_dir, f))]
generated_normal_files = [f for f in os.listdir(generated_normal_dir) if os.path.isfile(os.path.join(generated_normal_dir, f))]
print(len(generated_normal_files))
im = cv2.imread(os.path.join(generated_covid_dir,generated_covid_files[0]))
im.shape

1000


(150, 150, 3)

In [10]:
base_dir = '/content/covid19_radiography_data2/'
os.makedirs(base_dir,exist_ok=True)

train_dir = os.path.join(base_dir,'train')
os.makedirs(train_dir,exist_ok=True)
train_covid_dir = os.path.join(train_dir,'covid')
os.makedirs(train_covid_dir,exist_ok=True)
train_normal_dir = os.path.join(train_dir,'normal')
os.makedirs(train_normal_dir,exist_ok=True)

val_dir = os.path.join(base_dir,'val')
os.makedirs(val_dir,exist_ok=True)
val_covid_dir = os.path.join(val_dir,'covid')
os.makedirs(val_covid_dir,exist_ok=True)
val_normal_dir = os.path.join(val_dir,'normal')
os.makedirs(val_normal_dir,exist_ok=True)

test_dir = os.path.join(base_dir,'test')
os.makedirs(test_dir,exist_ok=True)
test_covid_dir = os.path.join(test_dir,'covid')
os.makedirs(test_covid_dir,exist_ok=True)
test_normal_dir = os.path.join(test_dir,'normal')
os.makedirs(test_normal_dir,exist_ok=True)

train_covid_files = allfiles_covid[:2170]
train_normal_files = allfiles_normal[:2170]
train_gencovid_files = generated_covid_files[:600]#g
train_gennormal_files = generated_normal_files[:600]#g
for f in train_covid_files:
    src = os.path.join(data_dir_covid, f)
    dst = os.path.join(train_covid_dir, f)
    shutil.copyfile(src, dst)
for f in train_normal_files:
    src = os.path.join(data_dir_normal, f)
    dst = os.path.join(train_normal_dir, f)
    shutil.copyfile(src, dst)
for f in train_gencovid_files:#g
    src = os.path.join(generated_covid_dir, f)
    dst = os.path.join(train_covid_dir, f)
    shutil.copyfile(src, dst)
for f in train_gennormal_files:#g
    src = os.path.join(generated_normal_dir, f)
    dst = os.path.join(train_normal_dir, f)
    shutil.copyfile(src, dst)

val_covid_files = allfiles_covid[2170:(2170+723)]
val_normal_files = allfiles_normal[2170:(2170+723)]
val_gencovid_files = generated_covid_files[600:800]#g
val_gennormal_files = generated_normal_files[600:800]#g
for f in val_covid_files:
    src = os.path.join(data_dir_covid, f)
    dst = os.path.join(val_covid_dir, f)
    shutil.copyfile(src, dst)
for f in val_normal_files:
    src = os.path.join(data_dir_normal, f)
    dst = os.path.join(val_normal_dir, f)
    shutil.copyfile(src, dst)
for f in val_gencovid_files:#g
    src = os.path.join(generated_covid_dir, f)
    dst = os.path.join(val_covid_dir, f)
    shutil.copyfile(src, dst)
for f in val_gennormal_files:#g
    src = os.path.join(generated_normal_dir, f)
    dst = os.path.join(val_normal_dir, f)
    shutil.copyfile(src, dst)


test_covid_files = allfiles_covid[(2170+723):(2170+723+723)]
test_normal_files = allfiles_normal[(2170+723):(2170+723+723)]
test_gencovid_files = generated_covid_files[800:]#g
test_gennormal_files = generated_normal_files[800:]#g
for f in test_covid_files:
    src = os.path.join(data_dir_covid, f)
    dst = os.path.join(test_covid_dir, f)
    shutil.copyfile(src, dst)
for f in test_normal_files:
    src = os.path.join(data_dir_normal, f)
    dst = os.path.join(test_normal_dir, f)
    shutil.copyfile(src, dst)
for f in test_gencovid_files:#g
    src = os.path.join(generated_covid_dir, f)
    dst = os.path.join(test_covid_dir, f)
    shutil.copyfile(src, dst)
for f in test_gennormal_files:#g
    src = os.path.join(generated_normal_dir, f)
    dst = os.path.join(test_normal_dir, f)
    shutil.copyfile(src, dst)
    


In [11]:
print(len(os.listdir(train_covid_dir)))
print(len(os.listdir(train_normal_dir)))
print(len(os.listdir(val_covid_dir)))
print(len(os.listdir(val_normal_dir)))
print(len(os.listdir(test_covid_dir)))
print(len(os.listdir(test_normal_dir)))

2770
2770
923
923
923
923


In [12]:
# generated_data_dir = '/content/gdrive/My Drive/Colab Notebooks/generated_imgs'
# len(os.listdir(os.path.join(generated_data_dir,'normal')))

In [13]:
# !mkdir -p /content/covid19_radiography_data/{test/{covid,normal},train/{covid,normal}}
# !find /content/covid19_radiography_data/COVID-19_Radiography_Dataset/COVID/ -maxdepth 1 -type f | head -3616 | xargs cp -t /content/covid19_radiography_data/train/covid/
# !find /content/covid19_radiography_data/COVID-19_Radiography_Dataset/Normal/ -maxdepth 1 -type f | head -4000 | xargs cp -t /content/covid19_radiography_data/train/normal/
## !find /content/covid19_radiography_data/COVID-19_Radiography_Dataset/COVID/ -maxdepth 1 -type f | tail -2500 | xargs cp -t /content/covid19_radiography_data/test/covid/
## !find /content/covid19_radiography_data/COVID-19_Radiography_Dataset/Normal/ -maxdepth 1 -type f | tail -2500 | xargs cp -t /content/covid19_radiography_data/test/normal/

In [14]:
# base_dir = '/content/covid19_radiography_data/'
# train_dir = os.path.join( base_dir, 'train')
##test_dir = os.path.join( base_dir, 'test')

In [15]:
train_datagen = ImageDataGenerator(rescale = 1.0/255.0)
                                   #validation_split=0.3)

val_datagen = ImageDataGenerator(rescale = 1.0/255.0)
test_datagen = ImageDataGenerator(rescale = 1.0/255.0)

train_generator = train_datagen.flow_from_directory(train_dir,
                                                    batch_size = 8,
                                                    class_mode = 'binary', 
                                                    target_size = (150, 150))#, color_mode='grayscale')
                                                    #subset='training')     

val_generator =  val_datagen.flow_from_directory(val_dir,
                                                 batch_size = 8,
                                                 class_mode = 'binary', 
                                                 target_size = (150, 150))#, color_mode='grayscale')

test_generator =  test_datagen.flow_from_directory(test_dir,
                                                   batch_size = 8,
                                                   class_mode = 'binary', 
                                                   target_size = (150, 150))#, color_mode='grayscale')
                                                   #subset='validation')

Found 5540 images belonging to 2 classes.
Found 1846 images belonging to 2 classes.
Found 1846 images belonging to 2 classes.


In [16]:
from tensorflow.keras.applications.inception_v3 import InceptionV3
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.applications import VGG16

# img_in = tf.keras.layers.Input(shape=(150,150,1))
# img_in_concat = tf.keras.layers.Concatenate()([img_in, img_in, img_in])

# pretrained_model = InceptionV3(
#     include_top=False, weights='imagenet',
#     input_shape=(150, 150, 3))

# pretrained_model = ResNet50(
#     include_top=False, weights='imagenet', 
#     input_shape=(150, 150, 3))

pretrained_model = VGG16(
    include_top=False, weights='imagenet', 
    input_shape=(150, 150, 3))

for layer in pretrained_model.layers:
  layer.trainable = False #change
  
#last_layer = pretrained_model.get_layer('mixed7') #InceptionV3
#last_layer = pretrained_model.get_layer('conv5_block3_out') #Resnet50 conv5_block3_out conv4_block6_out
last_layer = pretrained_model.get_layer('block5_pool') #VGG16
print('last layer output shape: ',last_layer.output_shape)
last_output = last_layer.output

58892288/58889256 [==============================] - 0s 0us/step
last layer output shape:  (None, 4, 4, 512)


In [17]:
x = layers.Flatten()(last_output)
x = layers.Dense(1024, activation='relu')(x)
x = layers.Dense(64, activation='relu')(x)                   
x = layers.Dense  (1, activation='sigmoid')(x)           

model = Model(pretrained_model.input, x) 

model.compile(optimizer = Adam(lr=0.001), #optimizer=Adam(learning_rate=0.0001)
              loss = 'binary_crossentropy', 
              metrics = ['accuracy'])

In [18]:
history = model.fit(
            train_generator,
            validation_data = val_generator,
            steps_per_epoch = len(train_generator),
            epochs = 3,
            validation_steps = len(val_generator),
            verbose = 1)

Epoch 1/3
693/693 [==============================] - 62s 41ms/step - loss: 0.4123 - accuracy: 0.8437 - val_loss: 0.1200 - val_accuracy: 0.9577
Epoch 2/3
693/693 [==============================] - 26s 38ms/step - loss: 0.1337 - accuracy: 0.9484 - val_loss: 0.1188 - val_accuracy: 0.9567
Epoch 3/3
693/693 [==============================] - 27s 39ms/step - loss: 0.1076 - accuracy: 0.9622 - val_loss: 0.1217 - val_accuracy: 0.9518


In [19]:
test_loss, test_acc = model.evaluate(test_generator)

231/231 [==============================] - 7s 29ms/step - loss: 0.1171 - accuracy: 0.9550
